This file details our efforts to compare a variety of different prompts and parameters in the use of a pretrained model to conduct few shot learning and generate a fake profile that sounds indistinguishable from other top matches.

Different avenues explored include using:
    - Full essays 
        - Example prompt:
            This is the input essay: 
            This is a good match:
            This is a good match:
            Write a new good match:
 
    - First x number of characters of an essay

    - Experiment with different parameters of the generate function
    
    - different/larger versions of model (distilgpt2, gpt2)



This file loads in the GPT2 model from the HuggingFace library, and uses most of an individuals essays, calculates their top matches, and constructs a prompt for a few shot encoder to produce a fake profile in the same style/tone that should also be a good match for them. It then implements an evaluation methodology by comparing where the fake profile would fall, relative to the rest of the input person's matches. 

In [1]:
!pip install sentence_transformers
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import pandas as pd
from scipy.spatial.distance import cosine
import numpy as np
!pip install ast
from ast import literal_eval

2024-03-04 17:07:10.188087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/0c/ytgqby892k1dcqcnn5gw20f00000gn/T/pip-install-tarf7l8v/ast_9383d1d257ce43a0b4e88d144af3404d/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 906, in open
      FileNotFoundError: [Errno 2] No such file or directory: '/private/var/folders/0c/ytgqby892k1dcqcnn5gw20f00000gn/T/pip-install-tarf7l8v/ast_9383d1d257ce43a0b4e88d144af3404d/AST/README'
      [end of output]
  
  note: This error originates from a

In [2]:
embedding_series = pd.read_csv('./embedding_series.csv').set_index('Unnamed: 0')
df_all = pd.read_csv('./okcupid_profiles.csv')
df_all = df_all.loc[embedding_series.index,:] 
matches = pd.read_csv('./okcupid_matches.csv').set_index('Unnamed: 0')

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/amaribauer/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [21]:
def rank_new_input(input_str, df_all, model, embedding_series, eval_fake = False, pref_gender=False, pref_age_lower=False, pref_age_higher=False, min_similarity_score = 0.5):
    df_possible = df_all.copy()
    if pref_gender:
        df_possible = df_possible.loc[df_possible.loc[:,'sex'] == pref_gender, :]
    if pref_age_higher:
        df_possible = df_possible[df_possible.loc[:, "age"] <= pref_age_higher]
    if pref_age_lower:
        df_possible = df_possible[df_possible.loc[:, "age"] >= pref_age_lower]
    user_embeddings = model.encode(input_str)
    if eval_fake:
        #ADD FAKE PROFILE STRING TO EMBEDDING SERIES and df_possible so that it will rank accordingly
        embedding_series.loc[99999, 'embedding'] = str(model.encode(eval_fake).tolist())
        fake_profile_row = pd.DataFrame([np.nan] * len(df_possible.columns)).T
        fake_profile_row.index = [99999]
        df_possible = pd.concat([df_possible, fake_profile_row]) 

    other_embeddings = [literal_eval(embedding_series.loc[i,'embedding']) for i in df_possible.index]
    # Compute the cosine similarity between the user's weighted embedding vector and all possible matches
    cosine_similarities = compute_cosine_similarity(user_embeddings, other_embeddings)
    # Recover index to match back to original dataframe
    similarity_scores = [(df_possible.index[index], score) for index, score in enumerate(cosine_similarities) if score >= min_similarity_score and score != 1]
    # Sort by similarity
    ranked_similarity = sorted(similarity_scores, key = lambda x: x[1], reverse = True)
    return ranked_similarity

def compute_cosine_similarity(target_vector, vectors):
    similarities = []
    for vector in vectors:
        similarity = 1 - cosine(target_vector, vector)  # 1 - cosine distance to get cosine similarity
        similarities.append(similarity)
    return similarities

In [10]:
def construct_prompt(input_string, matches, num_char = False):
    slice_len = min(2, len(matches))
    top_matches_slice = matches[:slice_len]
    essays_to_use = ["essay0", "essay1", "essay2", "essay3", "essay4", 
                   "essay5", "essay6", "essay7", "essay8"]
    prompt = 'Write a dating profile that would be a good match for the input person ' + "Input: " + input_string
    for i, val in top_matches_slice:
        essays_subset = df_all.loc[i,essays_to_use]
        output = essays_subset.str.cat()
        if num_char:
            output = output[:num_char]
        prompt = prompt  + " This is a good match: " + output
    return prompt + f"Input: {input_string}" + " Here is a new good match: "
    

In [7]:
new_input_matches = rank_new_input(input_str, False, 'f', 30,40, 0.2)
#returns list of tuples
new_input_matches #These are the top matches in the OKCupid dataframe for the new person. 
#We want our model to write a profile that ends up high here.
#TESTING OUT WITH AN ESSAY INPUT
input_str = """Hey there! I'm Jimmy, but you can call me Jim. 
I'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, 
but by night, I'm a dreamer exploring the wonders of the world, both near and far."""

prompt = construct_prompt(input_str, new_input_matches, 200) 
prompt_full = construct_prompt(input_str, new_input_matches) 
prompt

'Write a dating profile that would be a good match for the input person Input: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'m a curious soul with a zest for life and a passion for adventure. By day, I\'m a data scientist, \nbut by night, I\'m a dreamer exploring the wonders of the world, both near and far. This is a good match: ah, what can i say... i guess i am a curious person. i like learning and exploring new technologies, views, things and people. a current interest of mine is the culinary world in the east bay "why?\' b This is a good match: by answering a host of questions, i am curious to see the type of person i am supposedly \'compatible\' with on this website....will it be similar to my encounters in real life??!  i am unconventional, Input: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'m a curious soul with a zest for life and a passion for adventure. By day, I\'m a data scientist, \nbut by night, I\'m a dreamer exploring the wonders of the world, both near a

In [9]:
generator = pipeline('text-generation', model='gpt2')
all_returned = generator(prompt, do_sample=True, temperature = 0.9, truncation = True,
                         min_length=200, max_length = 1000, num_return_sequences=1)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [11]:
all_returned2 = generator(prompt_full, do_sample=True, temperature = 0.9, truncation = True,
                         min_length=200, max_length = 1000, num_return_sequences=1)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [23]:
print(all_returned)


[{'generated_text': 'Write a dating profile that would be a good match for the input person Input: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'m a curious soul with a zest for life and a passion for adventure. By day, I\'m a data scientist, \nbut by night, I\'m a dreamer exploring the wonders of the world, both near and far. This is a good match: down to earth, mellow while being witty and sarcastic. i can have just as much fun staying in and cooking a great meal as going out. i like good wine, anything to do with art and design, non judgmenta This is a good match: i have been called a marshmallow one that likes to be made all warm and gewie, a big teddy bear, amongst other things....haha. im a total sweetheart and would like to fall in love again someday.enjoyiInput: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'m a curious soul with a zest for life and a passion for adventure. By day, I\'m a data scientist, \nbut by night, I\'m a dreamer exploring the wonders of the w

In [13]:
print(all_returned2)

[{'generated_text': "Write a dating profile that would be a good match for the input person Input: Hey there! I'm Jimmy, but you can call me Jim. \nI'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, \nbut by night, I'm a dreamer exploring the wonders of the world, both near and far. This is a good match: down to earth, mellow while being witty and sarcastic. i can have just as much fun staying in and cooking a great meal as going out. i like good wine, anything to do with art and design, non judgmental people, laughing and working out to name a few...baking.on this website: you may notice i'm not sure it is for me... thought i might give it a shot. putting my toes in the water... This is a good match: i have been called a marshmallow one that likes to be made all warm and gewie, a big teddy bear, amongst other things....haha. im a total sweetheart and would like to fall in love again someday.enjoying it!!!!being normali have been told sev

In [10]:
fake_profile = all_returned[0]['generated_text'].replace(prompt, "")
#This is just the generated essay.
#THIS IS WHAT NEEDS TO BE CHECKED FOR TOXICITY
fake_profile

'\nI\'m getting a lot of requests for info regarding food and drink. I\'m trying on different clothes to show it off and I\'m not getting all that much of an interest in food. I\'ve read the comment section and found that it\'s pretty interesting and pretty much makes sense. \nI have a little extra money from the site, but this is better for my job, as you can see, this is what my website looks like. Input: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'ve got a lot of information on beer and stuff. I\'m trying on some clothes for the kids and it looks cool. \nI think I could use some pictures of my work as well..... Input: Hey there! I\'m Jimmy, but you can call me Jim. \nThanks for reading. I\'ve just seen your page here and my husband likes the look of it, sandwich making and I like the way it looks. Input: Hey there! I\'m Jimmy, but you can call me Jim. \nI\'m a curious soul with a zest for life and a passion for adventure. By day, i\'m a data scientist, \nbut by night, i\'m 

In [ ]:
#EVALUATION METHODOLOGY
new_output_matches = rank_new_input(input_str, fake_profile, 'f', 30,40, 0)
new_output_matches
#Theoretically, our fake profile, 99999, should be high here
for count, (i, sim) in enumerate(new_output_matches):
    if i == 99999:
        print(f"Our fake profile ranked {count}, out of {len(new_output_matches)} with a similarity score of {sim}")

In [22]:
#COLLATED FINAL FUNCTION
def generate_profile(input_str, eval_fake = False, pref_gender=False, pref_age_lower=False, 
                     pref_age_higher=False, min_similarity_score = 0.5 ):
    embedding_series = pd.read_csv('./embedding_series.csv').set_index('Unnamed: 0')
    df_all = pd.read_csv('./okcupid_profiles.csv')
    df_all = df_all.loc[embedding_series.index,:] 
    matches = pd.read_csv('./okcupid_matches.csv').set_index('Unnamed: 0')
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    new_input_matches = rank_new_input(input_str, df_all, model, embedding_series, eval_fake, pref_gender,   #need to pull in this function
                    pref_age_lower, pref_age_higher, min_similarity_score)
    prompt = construct_prompt(input_str, new_input_matches, 200)        #need to pull in this function
    
    generator = pipeline('text-generation', model='gpt2')
    all_returned = generator(prompt, do_sample=True, temperature = 0.9, truncation = True,
                         min_length=200, max_length = 1000, num_return_sequences=1)
    fake_profile = all_returned[0]['generated_text'].replace(prompt, "")
    #THIS IS WHAT NEEDS TO BE CHECKED FOR TOXICITY
    return fake_profile


In [23]:
generate_profile("""Hey there! I'm Jimmy, but you can call me Jim. 
I'm a curious soul with a zest for life and a passion for adventure. By day, I'm a data scientist, 
but by night, I'm a dreamer exploring the wonders of the world, both near and far.""")

/Users/amaribauer/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\nMy first date has been in a bar. \nIn the last couple of years, our relationship has been quite cordial. We both liked the fact that we would have had sex before the party. \nNow I am about to get caught up in some other, more serious sexual activities (with two guys) that are just too exciting. \nThe rest of us are more excited about this and that, so we are trying to figure out where things stand now. The next couple of months are going to be about both of us, so please stay tuned to the social media for the more exciting things that are to come and what they mean for our lives as we go along: \xa0– Don\'t Miss Us - \xa0– You Don\'t Have to Be a Sexed Guy to Want to be a Teenage Sex Worker!! \n– Don\'t Miss Him - A Conversation from a Different Kind of Body - \nHere is another big chance to meet up with a big, sexy woman who has been together three years and who has finally put a big grin on our faces. \nThe first two times he has not been in the same room, but just spent our even